<div align="center"> #----- SISTEMAS DE RECOMENDACIÓN -----# </div>

<img src="sistemas.png">

 ## **Trabajo Final de Data Science**

In [5]:
################################
#----- Carga de Librerías -----#
################################

!python -m spacy download en
!python -m spacy download es
!pip install -U spacy
!python -m spacy download en_core_web_sm

## Genéricas ##
import numpy as np
import pandas as pd
import datetime as dt
import re
import warnings
warnings.filterwarnings('ignore')

## Visualización
from plotnine import *
import folium
import seaborn as sns
from IPython.display import Image, HTML
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator

## Análisis de Datos - Modelamiento
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from langdetect import detect
import nltk
from nltk.corpus import stopwords
import plotly.express as px
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
import en_core_web_sm
from ast import literal_eval
from string import punctuation

     |████████████████████████████████| 12.0MB 2.6MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp37-none-any.whl size=12011740 sha256=6da9d3fe25a95545aa698f762cffd529e9d09da97d0d42964e84125b36c359cc
  Stored in directory: /private/var/folders/jc/lr5522ss7nn97zq58kg3xnph0000gp/T/pip-ephem-wheel-cache-qnlh_f92/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## * Evaluación del Modelo :

In [6]:
# Carga del csv 

listings_recomen = pd.read_csv('listings_recomen.csv', sep = ",")

In [7]:
listings_recomen['id'] = listings_recomen['id'].astype(str)

In [8]:
listings_recomen = listings_recomen.drop(['Unnamed: 0'], axis = 1)

In [9]:
listings_recomen.head(2)

,id,listing_url,name,space,summary,description,picture_url,host_id,host_is_superhost,host_url,...,name_low,description_low,space_low,listing_id,comments,compuesto,compuesto_var,sentimiento,adjetivos,content
0,23197,https://www.airbnb.com/rooms/23197,Forum DeLuxe 5 mins Walk To CCIB Center & Sea★,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,I do not accept groups of young people under 2...,https://a0.muscache.com/im/pictures/738532/806...,90417,t,https://www.airbnb.com/users/show/90417,...,forum deluxe mins walk ccib center sea,accept groups young people apartment suitable ...,elegant spacious apartment suitable ample loun...,23197.0,13.0,0.929346,0.072896,positivo,"huge, little, chilly, early, close, wonderful,...",forum deluxe mins walk ccib center sea // acc...
1,25786,https://www.airbnb.com/rooms/25786,NICE ROOM AVAILABLE IN THE HEART OF GRACIA,Room available for rent.- PEDRO PEREZ. Share...,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,JUST GO THROUGH THE MANY REVIEWS I GOT THROUGH...,https://a0.muscache.com/im/pictures/6619f0c7-8...,108310,t,https://www.airbnb.com/users/show/108310,...,nice room available heart gracia,go many reviews got years better feedback welc...,room available rent pedro perez shared catalan...,25786.0,5.0,0.000000,-0.888359,neutro,NaN,nice room available heart gracia // go many re...


In [10]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0.1, stop_words = 'english')
tfidf_matrix = tf.fit_transform(listings_recomen['content'])

In [ ]:
# toma su tiempo
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarities

In [ ]:
results = {}
for idx, row in listings_recomen.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], listings_recomen['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]
    
results

In [15]:
######################################
#-----    Rutinas Definidas     -----#
#####################################
def item(id):
    name   = listings_recomen.loc[listings_recomen['id'] == id]['content'].tolist()[0].split(' // ')[0]
    desc   = ' Descripción: ' + listings_recomen.loc[listings_recomen['id'] == id]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    rat    = ' Rating: ' + listings_recomen.loc[listings_recomen['id'] == id]['review_scores_rating'].astype(str)
    var    = ' Variación: ' + listings_recomen.loc[listings_recomen['id'] == id]['compuesto_var'].astype(str)
    sent   = ' Sentimiento: ' + listings_recomen.loc[listings_recomen['id'] == id]['sentimiento']
    ciud   = 'Ciudad: ' + listings_recomen.loc[listings_recomen['id'] == id]['city'] 
    prediction = str(name)  + str(desc) + str(ciud) + str(sent)+ str(rat) + str(var)
    return prediction

def recommend(item_id, num, ciudad):                                                                                   
    print('Sitio de inicio ' + str(num) + ' producto similar a ' + item(item_id))
    print('---------------------------------------------------------------------------')
    recs = results[item_id]
    tmp = []
    for rec in recs:
        if (listings_recomen[listings_recomen['id'] == (rec[1])].city.values != listings_recomen[listings_recomen['id'] == item_id].city.values) & (listings_recomen[listings_recomen['id'] == (rec[1])].city.values == ciudad):
            print(" Recommended: " + item(rec[1]) + "  (score:" + str(rec[0]) + ')')
            tmp.append(rec[1])
        if len(tmp)>= num:
            break

## * Visualización del recomendador :

In [16]:
# Salida del recomendador

recommend(item_id = '31184437' , num = 4, ciudad = 'Barcelona')

Sitio de inicio 4 producto similar a entire stfl flat brbalconyterpool view Descripción: licence tasteful bright fresh apartment villa alexia nd floor apt balcony terrace overlooking swimming pool set complex apartments charming puerto pollen a property ...60484    Ciudad: Mallorca
Name: city, dtype: object60484     Sentimiento: positivo
Name: sentimiento, dtype: object60484     Rating: 100.0
Name: review_scores_rating, dtype: object60484     Variación: nan
Name: compuesto_var, dtype: object
---
 Recommended: bcn sagrada familia penthouse Descripción: large bright apartment superb terrace view emblematic sagrada familia completely renovated early apartment ideally located steps sagrada familia avenue gaudi large b...4651    Ciudad: Barcelona
Name: city, dtype: object4651     Sentimiento: positivo
Name: sentimiento, dtype: object4651     Rating: 85.0
Name: review_scores_rating, dtype: object4651     Variación: 0.2037368421052632
Name: compuesto_var, dtype: object  (score:0.6627231111712